In [28]:
import argparse
import sys
import os
import torch
import yaml
import wandb
import random
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from EquivariantGraphNeuralNetwork import EquivariantGNN
from diffusion_x_h import E3DiffusionProcess
from split_to_train_and_test import SetUpData
from DataPreprocessor import SpectrumCompressor
from schedulefree import RAdamScheduleFree

sys.path.append('parts/')
from train_per_iretation import diffuse_as_batch, train_epoch, eval_epoch, generate, EarlyStopping
from loss_calculation import kabsch_torch
from def_for_main import load_model_state, evaluate_by_rmsd, noise_schedule_for_GammaNetwork, evaluate_by_rmsd_and_atom_type_eval, define_optimizer

In [19]:
project_name = str(input('Enter the name of the project: '))
run_id = str(input('Enter the run id: '))
api = wandb.Api()
run = api.run(f'{project_name}/{run_id}')
prms = run.config


In [20]:
#パラメータの設定
#全体のパラメータ
conditional=prms['conditional']
seed = prms['seed']
num_epochs = prms['num_epochs']
batch_size = prms['batch_size']
#optimizerのパラメータ
lr = prms['lr']
weight_decay = prms['weight_decay']
optim_type = prms['optimizer']
#early stoppingのパラメータ
patience = prms['patience']
#diffusion_processのパラメータ
num_diffusion_timestep = prms['num_diffusion_timestep']
noise_schedule = prms['noise_schedule']
noise_precision = prms['noise_precision']
power = prms['noise_schedule_power']
#spectrum_compressorのパラメータ
to_compress_spectrum = prms['to_compress_spectrum']
compressor_hidden_dim = prms['compressor_hidden_dim']
compressed_spectrum_size = prms['compressed_spectrum_size']
#egnnのパラメータ
L = prms['L']
atom_type_size = prms['atom_type_size']
spectrum_size = prms['spectrum_size']
d_size = prms['d_size']
t_size = prms['t_size']
exO_size = prms['exO_size']
if conditional:
    if prms['to_compress_spectrum']:
        h_size = atom_type_size + compressed_spectrum_size + t_size
    else:
        h_size = atom_type_size + spectrum_size + t_size
else:
    h_size = atom_type_size + t_size
if prms['give_exO']:
    h_size = h_size + exO_size
x_size = prms['x_size']
m_size = prms['m_size']   
m_input_size = h_size + h_size + d_size
m_hidden_size = prms['m_hidden_size']
m_output_size = m_size
h_input_size = h_size + m_size
h_hidden_size = prms['h_hidden_size']
h_output_size = h_size 
x_input_size = h_size + h_size + d_size
x_hidden_size = prms['x_hidden_size']
x_output_size = 1
onehot_scaling_factor = prms['onehot_scaling_factor']

In [21]:
diffusion_process = E3DiffusionProcess(s=noise_precision,power=power,num_diffusion_timestep=num_diffusion_timestep,noise_schedule=noise_schedule)
egnn = EquivariantGNN(L,m_input_size,m_hidden_size,m_output_size,x_input_size,x_hidden_size,x_output_size,h_input_size,h_hidden_size,h_output_size)
if to_compress_spectrum:
    spectrum_compressor = SpectrumCompressor(spectrum_size,compressor_hidden_dim,compressed_spectrum_size)
#使用するモデルをまとめた辞書nn_dictを定義
if to_compress_spectrum:
    nn_dict = {'egnn':egnn,'spectrum_compressor':spectrum_compressor}
else:
    nn_dict = {'egnn':egnn,'spectrum_compressor':None}

In [22]:
model_save_path = prms['model_save_path']
load_model_state(nn_dict,model_save_path,prms)

{'egnn': EquivariantGNN(
   (egcl_list): ModuleList(
     (0-4): 5 x EGCL()
   )
 ),
 'spectrum_compressor': None}

In [ ]:
project = 'amorphous'
run_name = str(input(f'Enter the run name ({run_id}): '))
run = wandb.init(project=project,name=run_name,config=prms)
dataset_path = str(input('Enter the path of the dataset: '))
dataset = torch.load(dataset_path)
wandb.config.update({'amorphous_dataset_path': dataset_path})
torch.set_default_tensor_type(torch.cuda.FloatTensor)
original_graph_list, generated_graph_list = generate(nn_dict,dataset,prms,diffusion_process)
generated_graph_save_path = os.path.join(wandb.run.dir,'generated_graph.pt')
original_graph_save_path = os.path.join(wandb.run.dir,'original_graph.pt')
torch.save(generated_graph_list,generated_graph_save_path)
torch.save(original_graph_list,original_graph_save_path)
wandb.config.update({'original_graph_save_path': original_graph_save_path},allow_val_change=True)
print('The original graph has been saved.')
wandb.config.update({'generated_graph_save_path': generated_graph_save_path},allow_val_change=True)
print('The generated graph has been saved.')
original_graph_list = torch.load(original_graph_save_path)
generated_graph_list = torch.load(generated_graph_save_path)
Si_tensor = torch.tensor([0,1],dtype=torch.long)
O_tensor = torch.tensor([1,0],dtype=torch.long)
density_O_original = []
density_O_generated = []
for i in range(len(original_graph_list)):
    num_O = 0
    original_graph = original_graph_list[i]
    generated_graph = generated_graph_list[i][-1]
    for j in range(original_graph.x.shape[0]):
        if torch.equal(original_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(original_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_original.append(num_O/original_graph.x.shape[0])
    num_O = 0
    for j in range(generated_graph.x.shape[0]):
        if torch.equal(generated_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(generated_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_generated.append(num_O/generated_graph.x.shape[0])
fig, ax = plt.subplots()
ax.plot([0,1],[0,1],linestyle='-',color='red')
ax.plot(density_O_original,density_O_generated,linestyle='None',marker='o')
ax.set_xlabel('density of O in original')
ax.set_ylabel('density of O in generated')
ax.set_title('density of O in original and generated')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
plt.text(0.05, 0.95, f'accuracy: {sum([1 for i in range(len(density_O_original)) if abs(density_O_original[i]-density_O_generated[i])==0])/len(density_O_original)}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
wandb.log({'atom_type_eval':wandb.Image(fig)})
plt.close()

wandb.finish()


/tmp/ipykernel_12549/2334357041.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dataset_path)
100%|██████████| 144/144 [1:23:08<00:00, 34.64s/it]


The original graph has been saved.
The generated graph has been saved.


In [1]:
run = wandb.init(project='amorphous',id='1am4buf5',resume='must')
config = run.config
original_graph_save_path = config['original_graph_save_path']
generated_graph_save_path = config['generated_graph_save_path']

original_graph_list = torch.load(original_graph_save_path)
generated_graph_list = torch.load(generated_graph_save_path)
Si_tensor = torch.tensor([0,1],dtype=torch.long)
O_tensor = torch.tensor([1,0],dtype=torch.long)
density_O_original = []
density_O_generated = []
for i in range(len(original_graph_list)):
    num_O = 0
    original_graph = original_graph_list[i]
    generated_graph = generated_graph_list[i][-1]
    for j in range(original_graph.x.shape[0]):
        if torch.equal(original_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(original_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_original.append(num_O/original_graph.x.shape[0])
    num_O = 0
    for j in range(generated_graph.x.shape[0]):
        if torch.equal(generated_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(generated_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_generated.append(num_O/generated_graph.x.shape[0])
fig, ax = plt.subplots()
ax.plot([0,1],[0,1],linestyle='-',color='red')
ax.plot(density_O_original,density_O_generated,linestyle='None',marker='o')
ax.set_xlabel('density of O in original')
ax.set_ylabel('density of O in generated')
ax.set_title('density of O in original and generated')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
plt.text(0.05, 0.95, f'accuracy: {sum([1 for i in range(len(density_O_original)) if abs(density_O_original[i]-density_O_generated[i])==0])/len(density_O_original)}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
wandb.log({'atom_type_eval':wandb.Image(fig)})
plt.close()

wandb.finish()

NameError: name 'wandb' is not defined

In [11]:
import argparse
import sys
import os
import torch
import yaml
import wandb
import random
import datetime
import pytz
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
from EquivariantGraphNeuralNetwork import EquivariantGNN
from diffusion_x_h import E3DiffusionProcess
from split_to_train_and_test import SetUpData
from DataPreprocessor import SpectrumCompressor
from schedulefree import RAdamScheduleFree

sys.path.append('parts/')
from train_per_iretation import diffuse_as_batch, train_epoch, eval_epoch, generate, EarlyStopping
from loss_calculation import kabsch_torch
from def_for_main import load_model_state, evaluate_by_rmsd, noise_schedule_for_GammaNetwork, evaluate_by_rmsd_and_atom_type_eval, define_optimizer

In [3]:
project_name = 'NN'
run_name = 'for alpha gen'

#parameterの読み込み
with open ('parameters.yaml','r') as file:
    prms = yaml.safe_load(file)
jst=pytz.timezone('Asia/Tokyo')
now = datetime.datetime.now(jst)
prms['now'] = now.strftime("%Y%m%d%H%M")

run = wandb.init(project=project_name,config=prms,name=run_name)



wandb: Currently logged in as: utokyo-okubo (utokyo-okubo-university-of-tokyo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [4]:
#パラメータの設定
#全体のパラメータ
conditional=prms['conditional']
seed = prms['seed']
num_epochs = prms['num_epochs']
batch_size = prms['batch_size']
#optimizerのパラメータ
lr = prms['lr']
weight_decay = prms['weight_decay']
optim_type = prms['optimizer']
#early stoppingのパラメータ
patience = prms['patience']
#diffusion_processのパラメータ
num_diffusion_timestep = prms['num_diffusion_timestep']
noise_schedule = prms['noise_schedule']
noise_precision = prms['noise_precision']
power = prms['noise_schedule_power']
#spectrum_compressorのパラメータ
to_compress_spectrum = prms['to_compress_spectrum']
compressor_hidden_dim = prms['compressor_hidden_dim']
compressed_spectrum_size = prms['compressed_spectrum_size']
#egnnのパラメータ
L = prms['L']
atom_type_size = prms['atom_type_size']
spectrum_size = prms['spectrum_size']
d_size = prms['d_size']
t_size = prms['t_size']
exO_size = prms['exO_size']
if conditional:
    if prms['to_compress_spectrum']:
        h_size = atom_type_size + compressed_spectrum_size + t_size
    else:
        h_size = atom_type_size + spectrum_size + t_size
else:
    h_size = atom_type_size + t_size
if prms['give_exO']:
    h_size = h_size + exO_size
x_size = prms['x_size']
m_size = prms['m_size']   
m_input_size = h_size + h_size + d_size
m_hidden_size = prms['m_hidden_size']
m_output_size = m_size
h_input_size = h_size + m_size
h_hidden_size = prms['h_hidden_size']
h_output_size = h_size 
x_input_size = h_size + h_size + d_size
x_hidden_size = prms['x_hidden_size']
x_output_size = 1
onehot_scaling_factor = prms['onehot_scaling_factor']

In [6]:
#default_tensor_typeの設定
torch.set_default_tensor_type(torch.cuda.FloatTensor)

#seedの設定
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

#modelの定義
diffusion_process = E3DiffusionProcess(s=noise_precision,power=power,num_diffusion_timestep=num_diffusion_timestep,noise_schedule=noise_schedule)
egnn = EquivariantGNN(L,m_input_size,m_hidden_size,m_output_size,x_input_size,x_hidden_size,x_output_size,h_input_size,h_hidden_size,h_output_size)
if to_compress_spectrum:
    spectrum_compressor = SpectrumCompressor(spectrum_size,compressor_hidden_dim,compressed_spectrum_size)
early_stopping = EarlyStopping(patience=patience)
setupdata = SetUpData(seed=seed,conditional=conditional)  

/home/rokubo/miniconda3/envs/diffusion/lib/python3.8/site-packages/torch/__init__.py:955: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)


In [7]:
if to_compress_spectrum:
    nn_dict = {'egnn':egnn,'spectrum_compressor':spectrum_compressor}
else:
    nn_dict = {'egnn':egnn,'spectrum_compressor':None}

In [13]:
dataset_path = './dataset/2NN/dataset.pt'
dataset = torch.load(dataset_path)
wandb.config.update({'dataset_path':dataset_path})
for data in dataset: #spectrumサイズの設定（-1~19eVがデフォルト）
    spectrum = data.spectrum
    #spectrum = data.spectrum_raw
    resized_spectrum = spectrum[:,:spectrum_size]
    data.spectrum = resized_spectrum

# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 乱数生成器を作成し、CUDAデバイスに設定
generator = torch.Generator(device=device)
generator.manual_seed(seed)  # 任意のシード値を設定

#dataset内のグラフに孤立原子からなるグラフがある場合、削除
dataset = [data for data in dataset if data.pos.shape[0] > 1]

train_eval_data, test_data = [], []
for data in dataset:
    if 'mp-6930' in data.id:
        test_data.append(data)
    else:
        train_eval_data.append(data)
train_data_length = int(len(train_eval_data)*0.9)
eval_data_length = len(train_eval_data) - train_data_length
train_data, eval_data = random_split(train_eval_data,[train_data_length,eval_data_length],generator=generator)
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,generator=generator)
eval_loader = DataLoader(eval_data,batch_size=batch_size,shuffle=True,generator=generator)

optimizer = define_optimizer(prms,nn_dict,diffusion_process,optim_type=optim_type)


/tmp/ipykernel_27179/2433349583.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(dataset_path)


In [14]:
for epoch in range(num_epochs):
    avg_loss_train = train_epoch(nn_dict,train_loader,prms,diffusion_process,optimizer)
    avg_loss_eval = eval_epoch(nn_dict,eval_loader,prms,diffusion_process,optimizer)

    #各epochにおけるlossを記録
    wandb.log({'train_loss':avg_loss_train,'eval_loss':avg_loss_eval})

    #early stopping
    if early_stopping.validate(avg_loss_eval):
        break

#モデルの保存
model_states = {'egnn':egnn.state_dict()}
if prms['to_compress_spectrum']:
    model_states['spectrum_compressor'] = spectrum_compressor.state_dict()
if prms['noise_schedule'] == 'learned':
    model_states['gamma'] = diffusion_process.gamma.state_dict()

model_save_path = os.path.join(wandb.run.dir,'model.pth')
torch.save(model_states,model_save_path)
wandb.config.update({'model_save_path':model_save_path})
print(f'model saved at {model_save_path}')
wandb.finish()

model saved at /jbod/rokubo/data/diffusion_model/wandb/run-20250306_114439-fv2pghsf/files/model.pth


eval_loss,█▅▇▂▄▂▅▅▄▄▂▃▁▃▄▄▁▃▄▂▄▄▅▆▄▄▄▄▅▅▅▆▅▆▇▄▄▅▅▅
train_loss,█▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval_loss,0.85389
train_loss,0.67535


In [ ]:
run = wandb.init(project='NN',id='fv2pghsf',resume='must')
original_graph_list, generated_graph_list = generate(nn_dict,test_data,prms,diffusion_process,gen_num_per_spectrum=300)
generated_graph_save_path = os.path.join(wandb.run.dir,'generated_graph.pt')
original_graph_save_path = os.path.join(wandb.run.dir,'original_graph.pt')
torch.save(generated_graph_list,generated_graph_save_path)
torch.save(original_graph_list,original_graph_save_path)
wandb.config.update({'original_graph_save_path': original_graph_save_path},allow_val_change=True)
print('The original graph has been saved.')
wandb.config.update({'generated_graph_save_path': generated_graph_save_path},allow_val_change=True)
print('The generated graph has been saved.')
original_graph_list = torch.load(original_graph_save_path)
generated_graph_list = torch.load(generated_graph_save_path)
Si_tensor = torch.tensor([0,1],dtype=torch.long)
O_tensor = torch.tensor([1,0],dtype=torch.long)
density_O_original = []
density_O_generated = []
for i in range(len(original_graph_list)):
    num_O = 0
    original_graph = original_graph_list[i]
    generated_graph = generated_graph_list[i][-1]
    for j in range(original_graph.x.shape[0]):
        if torch.equal(original_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(original_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_original.append(num_O/original_graph.x.shape[0])
    num_O = 0
    for j in range(generated_graph.x.shape[0]):
        if torch.equal(generated_graph.x[j],O_tensor):
            num_O += 1
        elif torch.equal(generated_graph.x[j],Si_tensor):
            pass
        else:
            print('Error')
    density_O_generated.append(num_O/generated_graph.x.shape[0])
fig, ax = plt.subplots()
ax.plot([0,1],[0,1],linestyle='-',color='red')
ax.plot(density_O_original,density_O_generated,linestyle='None',marker='o')
ax.set_xlabel('density of O in original')
ax.set_ylabel('density of O in generated')
ax.set_title('density of O in original and generated')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
plt.text(0.05, 0.95, f'accuracy: {sum([1 for i in range(len(density_O_original)) if abs(density_O_original[i]-density_O_generated[i])==0])/len(density_O_original)}', 
         transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
wandb.log({'atom_type_eval':wandb.Image(fig)})
plt.close()

wandb.finish()


  0%|          | 0/2055 [00:06<?, ?it/s]


KeyboardInterrupt: 

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7b095cf90220>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe